## 1. 데이터 준비

In [1]:
%matplotlib inline
# Standard library imports
import datetime
from datetime import datetime
import os
from copy import deepcopy
import pickle

# Third-party libraries imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
import seaborn as sns
import random
from itertools import combinations

# from IPython.display import clear_output
from scipy.spatial import distance
from sklearn.decomposition import PCA

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Subset
from torchvision import datasets, transforms

# Local application/library specific imports
from data_utils import (generate_server_idcs, CustomSubset, split_3class_unbalanced, split_contain_multiclass,
                        split_7plus3class_unbalanced, CombinedCustomSubset)
from fl_devices import Server, Client
from helper import ExperimentLogger, display_train_stats
from models import ConvNet, Representation, Ten_class_classifier


torch.manual_seed(42)
np.random.seed(42)

In [2]:
LOCAL_EPOCHS = 25
N_CLIENTS = 6
NUMBER_OF_CLUSTER = 3
ALPHA = 0.1
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
#data = datasets.CIFAR10(root="CIFAR10/", download=True)
data = datasets.MNIST(root="MNIST/", download=False)
idcs = np.random.permutation(len(data))

def cluster(server, clients, number_of_cluster):
    label_predicted = pd.DataFrame()
    # label_acc = pd.DataFrame()
    for i, client in enumerate(clients):
        pred = server.check_cluster(client.model)
        # print(f'pred: {pred}')
        label_predicted = pd.concat([label_predicted, pd.DataFrame(pred, index=[i])])
        # label_acc = pd.concat([label_acc, pd.DataFrame(acc, index=[i])])
    label_predicted.reset_index(drop=True, inplace=True)
    label_predicted.fillna(0, inplace=True)
    
    print(f'predicted label')
    print(label_predicted)

    cluster_idcs, number_of_cluster= server.cluster_clients(label_predicted)
    return label_predicted, cluster_idcs, number_of_cluster



def get_cluster_averages(df, cluster_idcs):
    cluster_averages = {}

    for i, cluster in enumerate(cluster_idcs):
        cluster_data = df.loc[cluster, :]
        cluster_average = cluster_data.mean()
        cluster_averages[i] = cluster_average

    cluster_averages_df = pd.DataFrame(cluster_averages).T

    # calculate percentage for each value in a row
    cluster_percentages_df = cluster_averages_df.div(cluster_averages_df.sum(axis=1), axis=0).multiply(100)

    # apply threshold and rounding
    cluster_percentages_df = cluster_percentages_df.where(cluster_percentages_df >= 5, 0).round()

    return cluster_percentages_df / 10


def get_cluster_weights(cluster_logits, cluster_weight_per_class):
    # Step 1: Find the class with the maximum value in each logit in cluster_logits
    # Assuming the dimension for classes is the second one
    max_classes = [torch.argmax(logit, dim=1) for logit in cluster_logits]
    
    print(max_classes)
    cluster_weights = []
    #c luster별로 
    for i, iogits in enumerate(cluster_logits):
        weights = []
        for j, logit in enumerate(logits):
            
            # print(f'max_classes: {max_classes[i][j]}')
            # print("Shape of cluster_weight_per_class[{}]: ".format(i), cluster_weight_per_class[i].shape)
            weights.append(max_classes[i][j])
        print(weights)
        cluster_weights.append(weights)

def visualize_clusters(label_predicted, real_cluster_distribution):
    # Reduce the dimension of the data
    pca = PCA(n_components=2)
    label_predicted_pca = pca.fit_transform(label_predicted)
    
    # Define colors for the clusters
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

    # Calculate the number of samples in each real cluster
    n_samples = len(label_predicted)
    real_cluster_sizes = [int(n_samples * dist) for dist in real_cluster_distribution]

    # Plot the clusters
    plt.figure(figsize=(8, 5))

    start_idx = 0  # To keep track of the start index of each real cluster
    for i, size in enumerate(real_cluster_sizes):
        end_idx = start_idx + size  # Calculate the end index for this real cluster
        color = colors[i % len(colors)]  # Determine the color for this real cluster

        # Scatter plot for points belonging to this real cluster
        for j in range(start_idx, end_idx):
            plt.scatter(label_predicted_pca[j, 0], label_predicted_pca[j, 1], c=color, s=50, label=f'Group {i+1}' if j == start_idx else "")

        start_idx = end_idx  # Update the start index for the next real cluster
    
    # Remove duplicate labels in legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), fontsize='xx-large')

    # Remove x and y ticks
    plt.xticks([])
    plt.yticks([])

    # Save the figure with narrow margins
    plt.subplots_adjust(left=0.01, right=0.99, top=0.99, bottom=0.01)
    current_time = datetime.now().strftime('%Y%m%d%H%M%S')
    plt.savefig(f'Figures/cluster_visualization_{current_time}.png', dpi=300, format='png')

    
    plt.show()



#     # Print the real cluster distribution
#     print("Real cluster distribution:", real_cluster_distribution)

#     # Calculate and print each client's cluster identity based on the real_cluster_distribution
#     n_clients = len(label_predicted)
#     cumulative_distribution = [0] + [sum(real_cluster_distribution[:i+1]) for i in range(len(real_cluster_distribution))]
#     client_cluster_id_real = [next((i for i, val in enumerate(cumulative_distribution) if val > client_idx / n_clients), -1) - 1 for client_idx in range(n_clients)]

#     print("Real cluster identity for each client:", client_cluster_id_real)

#     # Print each client's cluster identity based on the clusters argument
#     client_cluster_id_predicted = [next((i for i, cluster in enumerate(clusters) if client_idx in cluster), -1) for client_idx in range(n_clients)]
#     print("Predicted cluster identity for each client:", client_cluster_id_predicted)




def test_acc(server, clients, cluster_distribution):
    
    # Get individual client accuracies
    acc_clients = [client.evaluate() for client in clients]

    # Compute the average accuracy for each client
    client_acc = round(sum(acc_clients) / len(acc_clients), 3) if len(acc_clients) > 0 else 0

    # Compute cluster accuracies for this iteration
    cluster_accs_iteration = []
    start_idx = 0
    for distribution in cluster_distribution:
        end_idx = start_idx + int(distribution * len(clients))
        cluster_acc = round(sum(acc_clients[start_idx:end_idx]) / (end_idx - start_idx) + 0.000001, 3)
        cluster_accs_iteration.append(cluster_acc)
        start_idx = end_idx

    # Compute global accuracy for this iteration
    accuracies = [server.evaluate_distil(client.classifier) for client in clients]
    global_acc = round(np.mean(accuracies), 3)
    
    return client_acc, cluster_accs_iteration, global_acc


def get_global_logits(client_logits):
    avg_logits = torch.mean(torch.stack(client_logits), dim=0)
    return avg_logits

# def get_cluster_logits(client_logits, cluster_idcs):
#     cluster_logits = []
#     for i, cluster in enumerate(cluster_idcs):
#         cluster_client_logits = [client_logits[i] for i in cluster]
#         avg_cluster_logits = torch.mean(torch.stack(cluster_client_logits), dim=0)
#         cluster_logits.append(avg_cluster_logits)
#     return cluster_logits


def get_cluster_averages(df, cluster_idcs):
    cluster_averages = {}

    for i, cluster in enumerate(cluster_idcs):
        cluster_data = df.loc[cluster, :]
        cluster_average = cluster_data.mean()
        cluster_averages[i] = cluster_average

    cluster_averages_df = pd.DataFrame(cluster_averages).T

    # calculate percentage for each value in a row
    cluster_percentages_df = cluster_averages_df.div(cluster_averages_df.sum(axis=1), axis=0).multiply(100)

    # apply threshold and rounding
    cluster_percentages_df = cluster_percentages_df.where(cluster_percentages_df >= 5, 0).round()

    return cluster_percentages_df / 10

def compute_accuracy(global_logits, data_targets, server_idcs):
    """
    Compute accuracy of predictions based on global logits.

    Args:
    - global_logits (list of Tensors): Each Tensor represents the logits for a data instance, 
                                       and its size is the number of classes.
    - data_targets (list of ints): List of true labels for all data.
    - server_idcs (list of ints): Indices of data instances we're interested in.

    Returns:
    - accuracy (float): Computed accuracy.
    """
    true_labels = [data_targets[i] for i in server_idcs]
    
    correct_predictions = 0
    total_predictions = len(true_labels)

    for logits, label in zip(global_logits, true_labels):
        predicted_label = np.argmax(logits.cpu().numpy())  # Convert Tensor to numpy and get index of max value
        if predicted_label == label:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy


def save_tensor_to_csv(tensor, filename):
    # Convert tensor to numpy array
    array = tensor.cpu().detach().numpy()
    
    # Save numpy array to CSV file
    np.savetxt(filename, array, delimiter=",")

# def get_cluster_weights(cluster_logits, cluster_weight_per_class):
#     # Step 1: Find the class with the maximum value in each logit in cluster_logits
#     # Assuming the dimension for classes is the second one
#     max_classes = [torch.argmax(logit, dim=1) for logit in cluster_logits]
    
#     print(max_classes)
#     cluster_weights = []
#     #c luster별로 
#     for i, iogits in enumerate(cluster_logits):
#         weights = []
#         for j, logit in enumerate(logits):
            
#             # print(f'max_classes: {max_classes[i][j]}')
#             # print("Shape of cluster_weight_per_class[{}]: ".format(i), cluster_weight_per_class[i].shape)
#             weights.append(max_classes[i][j])
#         print(weights)
#         cluster_weights.append(weights)


### DS-FL

In [4]:
import torch
import torch.nn.functional as F
from scipy.special import kl_div

# Step 3: Apply softmax to get probability distribution
def apply_softmax(logits):
    return F.softmax(logits, dim=-1)

# Step 4: Average probability distributions
def average_prob_distributions(prob_distributions):
    return torch.mean(torch.stack(prob_distributions), dim=0)

# Step 5: Apply ERA technique to reduce entropy
def kl_divergence(p, q):
    return torch.sum(p * torch.log(p / q), dim=-1, keepdim=True)


def apply_ERA(avg_prob_distributions, local_prob_distributions, temperature=0.1):
    # Calculate reference distribution
    reference_distribution = apply_softmax(torch.mean(torch.stack(local_prob_distributions), dim=0))

    # Minimize KL divergence to reduce entropy, scaled by temperature
    kl_term = kl_divergence(avg_prob_distributions, reference_distribution)
    reduced_entropy_logits = avg_prob_distributions - temperature * kl_term

    return reduced_entropy_logits

# Step 1, 2, 3, 4, 5: Replace your existing function
def get_global_logits_DS_FL(client_logits, temperature=0.1):
    # Step 3: Apply softmax to each local logit
    client_prob_distributions = [apply_softmax(logits) for logits in client_logits]

    # Step 4: Average the probability distributions
    avg_prob_distributions = average_prob_distributions(client_prob_distributions)

    # Step 5: Apply ERA to the averaged logits
    reduced_entropy_logits = apply_ERA(avg_prob_distributions, client_prob_distributions, temperature)

    return reduced_entropy_logits.detach()

In [5]:
def make_global_logit(n_clients, total_client_data, distill_data, number_of_cluster, cluster_distribution, instance_per_class, instance_per_minor_class, instances_per_class_per_client):
    data_per_class=int(distill_data//10)
    train_idcs, test_idcs = idcs[:total_client_data], idcs[total_client_data:(total_client_data + int(distill_data * 2))]
    train_labels = data.targets
    test_labels = data.targets
    server_idcs = generate_server_idcs(test_idcs, test_labels, int(distill_data//10))
    print(f'sever idcs: {len(server_idcs)}')
    client_idcs, _ = split_7plus3class_unbalanced(train_idcs, train_labels, n_clients, cluster_distribution, instance_per_class, instance_per_minor_class)
    #client_idcs = split_contain_multiclass(train_idcs, train_labels, n_clients, instance_per_class, instances_per_class_per_client, cluster_distribution)
    
    client_data = [CustomSubset(data, idcs) for idcs in client_idcs]
    test_data = CustomSubset(data, server_idcs, transforms.Compose([transforms.ToTensor()]))
    print(f'test_data: {len(test_data)}')
    
    for i, client_datum in enumerate(client_data):
        client_datum.subset_transform = transforms.Compose([transforms.ToTensor()])

    server = Server(ConvNet, lambda x : torch.optim.Adam(x),test_data)

    clients = [Client(Representation, dat, i) 
           for i, dat in enumerate(client_data) if len(dat) > 20]
    
    # print(f'client count: {len(clients)}')

    client_logits = []
    
        
    # 1. Train classifier
    for i, client in enumerate(clients):
        client.classifier = Ten_class_classifier(client.model).to(device)
        client.train_classifier(lr=1e-3)

    client_acc, cluster_accs, global_acc = test_acc(server, clients, cluster_distribution)
    
    # 2. get client loigt
    for i, client in enumerate(clients):
        client_logits.append(server.get_clients_logit(client.classifier))

    # 3. make global logit
    global_logits = server.get_global_logits(client_logits)
    accuracy = compute_accuracy(global_logits, test_labels, server_idcs)
    
    print(f"Global Logit's Accuracy: {accuracy * 100:.2f}%")
    
    return server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution, client_acc, cluster_accs, global_acc

def distill_with_cluster_logit(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution, t):
    cluster_logits,  cluster_idcs, label_predicted = server.get_cluster_logits(client_logits, number_of_cluster, t)
    visualize_clusters(label_predicted, cluster_distribution)
    silhouette, ari = server.evaluate_clustering(label_predicted, cluster_distribution, cluster_idcs)
    
    print(f'silhouette: {silhouette}, ari: {ari}')

    cluster_distill_loaders = []
    
    for cluster_logit in cluster_logits:
        distill_loader = server.create_distill_loader(data, server_idcs, cluster_logit)
        cluster_distill_loaders.append(distill_loader)
        
    for i, client in enumerate(clients):
        if i % 10 == 0:
            print(f'client {i} distill')
        
        cluster_idx = next(j for j, cluster in enumerate(cluster_idcs) if i in cluster)
        
        client.distill((cluster_distill_loaders[cluster_idx]))
    
    client_acc, cluster_accs, global_acc = test_acc(server, clients, cluster_distribution)
    
    return client_acc, cluster_accs, global_acc, silhouette, ari


def get_cluster_acc(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution, t):
    cluster_logits,  cluster_idcs, label_predicted = server.get_cluster_logits(client_logits, number_of_cluster, t)
    visualize_clusters(label_predicted, cluster_distribution)
    silhouette, ari = server.evaluate_clustering(label_predicted, cluster_distribution, cluster_idcs)
    
    print(f'silhouette: {silhouette}, ari: {ari}')
    return silhouette, ari


def FedDF(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution):
    distill_loader = server.create_distill_loader(data, server_idcs, global_logits)
    
    for i, client in enumerate(clients):
        if i % 10 == 0:
            print(f'client {i} distill')
        client.distill(distill_loader)
    
    client_acc, cluster_accs, global_acc = test_acc(server, clients, cluster_distribution)
    
    return client_acc, cluster_accs, global_acc


def DS_FL(server, clients, client_logits, number_of_cluster, server_idcs, cluster_distribution):
    global_logits = get_global_logits_DS_FL(client_logits)

    distill_loader = server.create_distill_loader(data, server_idcs, global_logits)
    
    for i, client in enumerate(clients):
        if i % 10 == 0:
            print(f'client {i} distill')
        client.distill(distill_loader)
    
    client_acc, cluster_accs, global_acc = test_acc(server, clients, cluster_distribution)
    
    return client_acc, cluster_accs, global_acc

# def distill_with_dual_logit(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution, t):
#     global_distill_loader = server.create_distill_loader(data, server_idcs, global_logits)
    
#     cluster_logits,  cluster_idcs, label_predicted = server.get_cluster_logits(client_logits, number_of_cluster, t)
#     silhouette, ari = server.evaluate_clustering(label_predicted, cluster_distribution, cluster_idcs)
    
#     # print(f'silhouette: {silhouette}, ari: {ari}')

#     cluster_distill_loaders = []
    
#     for cluster_logit in cluster_logits:
#         dual_distill_loader = server.create_dual_distill_loader(data, server_idcs, cluster_logit, global_logits)
#         cluster_distill_loaders.append(dual_distill_loader)
        
#     for i, client in enumerate(clients):
#         if i % 10 == 0:
#             print(f'client {i} distill')
        
#         cluster_idx = next(j for j, cluster in enumerate(cluster_idcs) if i in cluster)
        
#         client.dual_distill((cluster_distill_loaders[cluster_idx]), silhouette)
    
#     client_acc, cluster_accs, global_acc = test_acc(server, clients, cluster_distribution)
    
#     return client_acc, cluster_accs, global_acc, silhouette, ari


def copy_classifiers(clients):
    classifiers_state = []
    for client in clients:
        classifier_state = client.classifier.state_dict()
        classifiers_state.append(classifier_state)
    return classifiers_state


In [6]:
def clustering_exp(n_clients, total_client_data, distill_data, number_of_cluster, cluster_distribution, instance_per_class, instance_per_minor_class, instances_per_class_per_client, t):
   
    client_accs = []
    cluster_accs = []
    global_accs = []
    
    (
        server, clients, client_logits, global_logits,
        number_of_cluster, server_idcs, cluster_distribution,
        new_client_acc, new_cluster_accs, new_global_acc
    ) = make_global_logit(
        n_clients, total_client_data, distill_data,
        number_of_cluster, cluster_distribution,
        instance_per_class, instance_per_minor_class, instances_per_class_per_client
    )

    
    print(f'acc before distill: {new_client_acc}, {new_cluster_accs}, {new_global_acc}')
    client_accs.append(new_client_acc)
    cluster_accs.append(new_cluster_accs)
    global_accs.append(new_global_acc)

    classifiers_state = copy_classifiers(clients)
    
    sil, ari = get_cluster_acc(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution, t)
    return sil, ari


def get_combination(array_length, class_per_cluster):
    if class_per_cluster == 1:
        combinations = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
    elif class_per_cluster == 2:
        combinations = [[0,1], [2,3], [4,5], [6,7], [8,9], [0,5], [1,6], [2,7], [3,8], [4,9], [0,3], [1,4], [2,5], [6,8], [7,9], [0,4], [1,5], [2,9], [3,7], [8,6]]
    elif class_per_cluster == 3:
        combinations = [[0,1,2], [3,4,5], [6,7,8], [9,0,1], [2,3,4], [5,6,7], [8,9,0], [1,2,3], [4,5,6], [7,8,9]]
    elif class_per_cluster == 4:
        combinations = [[0,1,2,3],[4,5,6,7],[8,9,0,1],[2,3,4,5],[6,7,8,9],[2,4,6,8],[1,3,5,9],[0,3,6,9],[1,4,7,0],[2,5,8,1]]
    elif class_per_cluster == 5:
        combinations = [[0,1,2,3,4],[4,5,6,7,8],[8,9,0,1,2],[2,3,4,5,6],[6,7,8,9,0],[2,4,6,8,0],[1,3,5,7,9],[0,2,3,6,9],[1,4,6,7,0],[2,5,6,8,1]]
    else:
        return "Invalid class_per_cluster"
    
    return combinations[:array_length]


In [7]:
def cluster_train_result_exp(n_clients, total_client_data, distill_data, number_of_cluster, cluster_distribution, instance_per_class, instance_per_minor_class, instances_per_class_per_client, t):
   
    client_accs = []
    cluster_accs = []
    global_accs = []
    
    (
        server, clients, client_logits, global_logits,
        number_of_cluster, server_idcs, cluster_distribution,
        new_client_acc, new_cluster_accs, new_global_acc
    ) = make_global_logit(
        n_clients, total_client_data, distill_data,
        number_of_cluster, cluster_distribution,
        instance_per_class, instance_per_minor_class, instances_per_class_per_client
    )

    
    print(f'acc before distill: {new_client_acc}, {new_cluster_accs}, {new_global_acc}')
    
    client_accs.append(new_client_acc)
    cluster_accs.append(new_cluster_accs)
    global_accs.append(new_global_acc)

    classifiers_state = copy_classifiers(clients)
    
    new_client_acc, new_cluster_accs, new_global_acc, silhouette, ari = distill_with_dual_logit(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution, t)
    
    client_accs.append(new_client_acc)
    cluster_accs.append(new_cluster_accs)
    global_accs.append(new_global_acc)
    
    for i, client in enumerate(clients):
        client.classifier.load_state_dict(classifiers_state[i])
    
    new_client_acc, new_cluster_accs, new_global_acc, silhouette, ari = distill_with_cluster_logit(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution, t)
    
    client_accs.append(new_client_acc)
    cluster_accs.append(new_cluster_accs)
    global_accs.append(new_global_acc)
    
    
    for i, client in enumerate(clients):
        client.classifier.load_state_dict(classifiers_state[i])
    
    new_client_acc, new_cluster_accs, new_global_acc = DS_FL(server, clients, client_logits, number_of_cluster, server_idcs, cluster_distribution)
    
    client_accs.append(new_client_acc)
    cluster_accs.append(new_cluster_accs)
    global_accs.append(new_global_acc)
    
    for i, client in enumerate(clients):
        client.classifier.load_state_dict(classifiers_state[i])
    
    new_client_acc, new_cluster_accs, new_global_acc = FedDF(server, clients, client_logits, global_logits, number_of_cluster, server_idcs, cluster_distribution)
    
    client_accs.append(new_client_acc)
    cluster_accs.append(new_cluster_accs)
    global_accs.append(new_global_acc)
    
    print(f'Dual: {client_accs[-4]}, {cluster_accs[-4]}, {global_accs[-4]}')
    print(f'Cluster: {client_accs[-3]}, {cluster_accs[-3]}, {global_accs[-3]}')
    print(f'DS-FL: {client_accs[-2]}, {cluster_accs[-2]}, {global_accs[-2]}')
    print(f'FedDF: {client_accs[-1]}, {cluster_accs[-1]}, {global_accs[-1]}')
    return client_accs, cluster_accs, global_accs, silhouette, ari


### Performance Exp

In [ ]:
import pandas as pd

t = 2
data_per_client = 500
group_count = 3
class_per_group = 3
instance_per_classes = [165, 158, 150, 133, 117, 100, 84]
instance_per_minor_classes = [0, 4, 7, 14, 21, 29, 36]
minor_class_ratios = [0, 5, 10, 20, 30, 40, 50]

all_results = []

for minor_class_ratio, instance_per_class, instance_per_minor_class in zip(minor_class_ratios, instance_per_classes, instance_per_minor_classes):

    # Update cluster_distribution based on new group_count
    cluster_distribution = [1/group_count for i in range(1, group_count+1)]
    
    # Initialize n_clients based on group_count
    n_clients = group_count * 5
    

    instances_per_class_per_client = get_combination(group_count, class_per_group)

    # Run the experiment
    client_accs, cluster_accs, global_accs, silhouette, ari = cluster_train_result_exp(
        n_clients, 40000, 400, group_count, cluster_distribution, instance_per_class, instance_per_minor_class, instances_per_class_per_client, t
    )

    # Append to the list as a dictionary
    all_results.append({
        'minor_class_ratio': minor_class_ratio,
        'group_count': group_count,
        'class_per_group': class_per_group,
        'client_accs': client_accs,
        'cluster_accs': cluster_accs,
        'global_accs': global_accs,
        'silhouette': silhouette,
        'ari': ari
    })

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(all_results)

sever idcs: 400
data_per_class_3: 84, data_per_class_7: 36
test_data: 400
Global Logit's Accuracy: 86.75%
acc before distill: 0.858, [0.872, 0.855, 0.846], 0.893
[2 2 2 2 2 1 0 1 1 1 0 0 0 0 0]
15
client 0 distill
client 10 distill
[2 2 2 2 2 1 0 1 1 1 0 0 0 0 0]
15
silhouette: 0.3211004137992859, ari: 0.7920792079207921
client 0 distill
client 10 distill
client 0 distill
client 10 distill
Dual: 0.897, [0.888, 0.893, 0.911], 0.927
Cluster: 0.9, [0.896, 0.896, 0.908], 0.911
FedDF: 0.884, [0.896, 0.872, 0.883], 0.927
sever idcs: 400
data_per_class_3: 165, data_per_class_7: 0
test_data: 400
Global Logit's Accuracy: 72.50%
acc before distill: 0.972, [0.974, 0.964, 0.979], 0.322
[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]
15
client 0 distill
client 10 distill


In [ ]:
df

In [ ]:
# df.to_csv('./results/large_group')